In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import random
import pandas as pd
from datetime import datetime, timedelta

In [8]:
# Atividades
ENTER_ACTIVITY = "Entrada no Estacionamento"
EXIT_ACTIVITY = "Saída do Estacionamento"
ACTIVITIES = ["Solicitar Ticket", "Scan Sem Parar", "Impressão do Ticket", "Scan Sem Parar", "Pagar Estacionamento", "Validar Ticket", "Perda de Ticket", "Reservar Vaga", "Cancelar Reserva", "Comprar Cartão Pré-pago", "Adicionar Créditos", "Cancelar Cartão Pré-pago", "Leitura do Cartão Pré-pago"]

# Regras de precedência
PRECEDENCE = {
    "Solicitar Ticket": [ENTER_ACTIVITY],
    "Scan Sem Parar": [ENTER_ACTIVITY],
    "Impressão do Ticket": ["Solicitar Ticket"],
    "Validar Ticket": ["Impressão do Ticket"],
    "Pagar Estacionamento": ["Validar Ticket"],
    "Cancelar Reserva": ["Reservar Vaga"],
    "Perda de Ticket": ["Impressão do Ticket"],
    "Cancelar Cartão Pré-pago": ["Comprar Cartão Pré-pago"],
    EXIT_ACTIVITY: ["Scan Sem Parar", "Validar Ticket", "Leitura do Cartão Pré-pago"],
}

# Regras de obrigação
OBLIGATION = {
    "Reservar Vaga": ["Cancelar Reserva", "Entrada no Estacionamento"],
    "Soliciar Ticket": ["Impressão do Ticket"],
}

# Regras de proibição
PROHIBITION = {
    "Adicionar Crédito": ["Cancelar Cartão Pré-pago"],
}

# Probabilidades das atividades (pesos)
ACTIVITY_PROBABILITY = {
    ENTER_ACTIVITY: 0,
    "Solicitar Ticket": 10,
    "Impressão do Ticket": 9.5,
    "Scan Sem Parar": 1,
    "Pagar Estacionamento": 8,
    "Validar Ticket": 8,
    "Perda de Ticket": 0.2,
    "Reservar Vaga": 1,
    "Cancelar Reserva": 0.4,
    "Comprar Cartão Pré-pago": 1.5,
    "Adicionar Créditos": 2,
    "Cancelar Cartão Pré-pago": 0.3,
    "Leitura do Cartão Pré-pago": 3,
    EXIT_ACTIVITY: 10,
}

ACTIVITY_DURATIONS = {
    ENTER_ACTIVITY: 0,
    "Solicitar Ticket": 10,
    "Impressão do Ticket": 10,
    "Scan Sem Parar": 2,
    "Pagar Estacionamento": 80,
    "Validar Ticket": 6,
    "Perda de Ticket": 0,
    "Reservar Vaga": 120,
    "Cancelar Reserva": 40,
    "Comprar Cartão Pré-pago": 180,
    "Adicionar Créditos": 40,
    "Cancelar Cartão Pré-pago": 480,
    "Leitura do Cartão Pré-pago": 2,
    EXIT_ACTIVITY: 0,
}

# Recursos disponíveis
RESOURCES = ["PCR8249", "RZQ8I54", "PCK0O98", "PAB8H80", "KHH6204", "SCR1905", "KHH6984", "KHI0043", "PCC1I09", "KIE0IOP", "JAC1990", "DOP0POP", "ASD4578", "ALK1L34", "KPA3B84", "PLL8G47", "PEC2468", "KHH4405", "KEA2308", "KED4O69"]

# Gera a próxima atividade aleatoriamente com base nas probabilidades
def get_next_activity(current_activity, case_activities, demanded_activities):
    all_possible_activities = ACTIVITIES + [EXIT_ACTIVITY]
    possible_next_activities = set()

    # Verifica as atividades que podem ser executadas em seguida, respeitando a precedência
    for activity in all_possible_activities:
        if activity == ENTER_ACTIVITY:
            continue
        if activity == current_activity:
            continue
        if activity in PRECEDENCE and any(dep in case_activities for dep in PRECEDENCE[activity]):
            possible_next_activities.add(activity)
        elif activity not in PRECEDENCE:
            possible_next_activities.add(activity)
        else:
            pass
    
    # Remove proíbições, caso haja alguma para o caso atual
    for done_activity in case_activities:
        if done_activity in PROHIBITION and any(dep in possible_next_activities for dep in PROHIBITION[done_activity]):
            possible_next_activities.remove(done_activity)

    # Seleciona aleatoriamente a próxima atividade com base nas probabilidades
    possible_next_activities = list(possible_next_activities)
    most_possible_next_activities = random.choices(possible_next_activities, weights=[ACTIVITY_PROBABILITY[activity] for activity in possible_next_activities])
    next_activity = most_possible_next_activities[0]

    if next_activity == EXIT_ACTIVITY and len(demanded_activities) > 0:
        current_demanded_activities = list(demanded_activities)
        next_activity = random.choices(current_demanded_activities, weights=[ACTIVITY_PROBABILITY[activity] for activity in current_demanded_activities])[0]
        demanded_activities.remove(next_activity)

    # Verifica as atividades obrigatórias
    if next_activity in OBLIGATION:
        obligations = OBLIGATION[next_activity]
        obl = random.choices(obligations, weights=[ACTIVITY_PROBABILITY[activity] for activity in obligations])[0]
        if obl not in case_activities:
            demanded_activities.add(obl)

    return next_activity

# Gera uma sequência de atividades para um caso, respeitando as regras de precedência e obrigações
def generate_activities():
    case_activities = [ENTER_ACTIVITY]
    demanded_activities = set()

    while True:
        current_activity = case_activities[-1]
        next_activity = get_next_activity(current_activity, case_activities, demanded_activities)
        case_activities.append(next_activity)
        if next_activity == EXIT_ACTIVITY:
            demanded_activities.clear()
            break

    return case_activities

# Gera o log sintético de atividades
def generate_log(num_cases):
    log_data = []
    resource = random.choice(RESOURCES)
    for i in range(num_cases):
        print(f'Gerando caso {i}')
        case_activities = generate_activities()
        start_time = datetime(2023, 7, 1, random.randint(7, 12), random.randint(0, 59), 0)
        for activity in case_activities:
            if activity == ENTER_ACTIVITY:
                resource = random.choice(RESOURCES)  # Escolhe um novo recurso para as atividades de entrada
            expected_duration = ACTIVITY_DURATIONS[activity]
            duration = random.randint(round(0.6 * expected_duration), round(2 * expected_duration))  # Duração aleatória em minutos
            end_time = start_time + timedelta(seconds=duration)
            log_data.append([i + 1, activity, resource, start_time.strftime("%Y-%m-%d %H:%M:%S"), end_time.strftime("%Y-%m-%d %H:%M:%S")])

            # Se atividades levarem a ficar no estabelecimento, demora um pouco mais para a próxima atividade
            if (activity == "Impressão do Ticket" or activity == "Scan Sem Parar"):
                start_time = end_time + timedelta(minutes=random.randint(0, 240))
            else:
                start_time = end_time + timedelta(seconds=random.randint(0, 60))
    return log_data

# Função para escrever o log em um arquivo CSV
def write_log_to_csv(df, filename):
    df.to_csv(filename, index=False)

In [9]:
if __name__ == "__main__":
    num_cases = 1000
    log_data = generate_log(num_cases)
    df = pd.DataFrame(log_data, columns=["Caso", "Atividade", "Recurso", "Início", "Fim"])
    print(df)

    # Escreve o log em um arquivo CSV chamado "log_estacionamento.csv"
    write_log_to_csv(df, "log_estacionamento.csv")

Gerando caso 0
Gerando caso 1
Gerando caso 2
Gerando caso 3
Gerando caso 4
Gerando caso 5
Gerando caso 6
Gerando caso 7
Gerando caso 8
Gerando caso 9
Gerando caso 10
Gerando caso 11
Gerando caso 12
Gerando caso 13
Gerando caso 14
Gerando caso 15
Gerando caso 16
Gerando caso 17
Gerando caso 18
Gerando caso 19
Gerando caso 20
Gerando caso 21
Gerando caso 22
Gerando caso 23
Gerando caso 24
Gerando caso 25
Gerando caso 26
Gerando caso 27
Gerando caso 28
Gerando caso 29
Gerando caso 30
Gerando caso 31
Gerando caso 32
Gerando caso 33
Gerando caso 34
Gerando caso 35
Gerando caso 36
Gerando caso 37
Gerando caso 38
Gerando caso 39
Gerando caso 40
Gerando caso 41
Gerando caso 42
Gerando caso 43
Gerando caso 44
Gerando caso 45
Gerando caso 46
Gerando caso 47
Gerando caso 48
Gerando caso 49
Gerando caso 50
Gerando caso 51
Gerando caso 52
Gerando caso 53
Gerando caso 54
Gerando caso 55
Gerando caso 56
Gerando caso 57
Gerando caso 58
Gerando caso 59
Gerando caso 60
Gerando caso 61
Gerando caso 62
Ge